In [2]:
import pandas as pd
import numpy as np
import pickle

# Import Dataframe

In [32]:
# Load pickled dataframe

file = 'pickles/features_04_30.pkl'

with open(file, 'rb') as f:
    df_features = pickle.load(f)

In [33]:
df_features.head()

,"(., '')","(PRP, VBD)",``,"(,, '')","('', PRP, VBD)","(,, ADP)","(NOUN, VBD)",VBD,"(VBD, .)","(VBD, ,)",...,"('', PRP)","(,, VBD)",source,readability_SMOG,avg_sent_len,Num_Sentences,sd_sent_len,norm_stop_freq,norm_punct_freq,norm_funct_freq
0,0.0,0.000000,0.000000,0.0,0.0,0.013333,0.013333,0.013158,0.0,0.0,...,0.0,0.0,economist,11.2,20.666667,3,11.841546,0.421875,0.234375,0.390625
1,0.0,0.000000,0.000000,0.0,0.0,0.013514,0.000000,0.013333,0.0,0.0,...,0.0,0.0,economist,10.7,17.000000,4,3.391165,0.500000,0.132353,0.470588
2,0.0,0.000000,0.000000,0.0,0.0,0.012346,0.000000,0.000000,0.0,0.0,...,0.0,0.0,economist,5.7,13.800000,5,5.844656,0.422535,0.197183,0.436620
3,0.0,0.009346,0.000000,0.0,0.0,0.000000,0.000000,0.018519,0.0,0.0,...,0.0,0.0,economist,11.6,17.800000,5,7.626270,0.422222,0.255556,0.411111
4,0.0,0.000000,0.013889,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,economist,13.6,18.666667,3,7.408704,0.362069,0.327586,0.344828


# A bunch of classifiers

In [27]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,f1_score,precision_score,recall_score
from xgboost import XGBClassifier

In [38]:
# prepare models
models = []
models.append(('LR', LogisticRegression(multi_class='multinomial',solver='lbfgs')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('XGBoost',XGBClassifier( n_estimators=1000)))
#models.append(('SVM', SVC(kernel = 'linear')))

In [34]:
# X -> features, y -> label
X=df_features.loc[:, df_features.columns != 'source']

#X=df_features[['readability_SMOG','avg_sent_len','sd_sent_len','norm_stop_freq','norm_punct_freq','norm_funct_freq']]
y=df_features.source
print(len(X))

52812


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.3, random_state = 0)

val_accuracy_all={}
val_fscore_all={}
val_precision_all={}
val_recall_all={}

train_accuracy_all={}
train_fscore_all={}
train_precision_all={}
train_recall_all={}

for name,model in models:
    print('Training')
    clf=model.fit(X_train,y_train)
    print('Testing')
    y_pred=clf.predict(X_test)
    print(name)
    val_precision_all[name]=precision_score(y_test,y_pred,average='macro')
    val_recall_all[name]=recall_score(y_test,y_pred,average='macro')
    val_fscore_all[name]=f1_score(y_test, y_pred, average='macro')
    val_accuracy_all[name]=accuracy_score(y_test, y_pred)

    y_train_pred=clf.predict(X_train)
    train_precision_all[name]=precision_score(y_train,y_train_pred,average='macro')
    train_recall_all[name]=recall_score(y_train,y_train_pred,average='macro')
    train_fscore_all[name]=f1_score(y_train,y_train_pred, average='macro')
    train_accuracy_all[name]=accuracy_score(y_train,y_train_pred)

val_classifier_results=pd.DataFrame({'Accuracy':pd.Series(val_accuracy_all),'Precision':pd.Series(val_precision_all),'Recall':pd.Series(val_recall_all),'F1_Score':pd.Series(val_fscore_all)}) 

train_classifier_results=pd.DataFrame({'Accuracy':pd.Series(train_accuracy_all),'Precision':pd.Series(train_precision_all),'Recall':pd.Series(train_recall_all),'F1_Score':pd.Series(train_fscore_all)})  



Training
Testing
LR
Training
Testing
LDA
Training
Testing
KNN
Training
Testing
CART
Training
Testing
NB
Training
Testing
XGBoost


In [40]:
print(train_classifier_results)

         Accuracy  F1_Score  Precision    Recall
CART     0.999540  0.999534   0.999514  0.999555
KNN      0.573171  0.566496   0.574451  0.578462
LDA      0.586372  0.579189   0.578624  0.581168
LR       0.513255  0.497557   0.500262  0.501655
NB       0.427586  0.410950   0.494164  0.448846
XGBoost  0.693681  0.687154   0.685473  0.690629


In [41]:
print(val_classifier_results)

         Accuracy  F1_Score  Precision    Recall
CART     0.501704  0.494884   0.495133  0.494662
KNN      0.382984  0.374119   0.373405  0.385623
LDA      0.586026  0.578886   0.577916  0.580324
LR       0.513065  0.495733   0.497527  0.500285
NB       0.429500  0.412919   0.492705  0.450919
XGBoost  0.629702  0.621602   0.619203  0.625379


# SVM

In [ ]:
# Grid search:

# vary kernels
# vary 'C'

# Record testing error AND training error

In [ ]:
#SVM with grid search. Needs to be completed
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['accuracy','precision_macro', 'recall_macro','f1_macro']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)



# Logistic Regression

In [ ]:
# Try different regularization techniques

# If it's not overfitting, maybe try adding in some interaction terms?

# XGBOOST

In [48]:
params = {'min_child_weight': [1, 5, 10],
        'gamma': [0,0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1, 0.2],
        }


gsearch1 = GridSearchCV(estimator = XGBClassifier(n_estimators=300,  
                                                  objective= 'multi:softmax',
                                                  num_class=4, 
                                                  nthread=4, seed=27), 
                        param_grid = params, 
                        scoring='accuracy',
                        n_jobs=4,
                        iid=False, 
                        cv=4)


In [ ]:
gsearch1.fit(X_train,y_train)

In [ ]:
#Observe The best params and substitute please?
gsearch1.best_params_, gsearch1.best_score_

In [1]:
params2 = {'n_estimators': [300, 600, 1000,1500,2000,3000]
        }
gsearch2 = GridSearchCV(estimator = XGBClassifier(#Add best params from gsearch1 here
                                                  objective= 'multi:softmax',
                                                  num_class=4, 
                                                  nthread=4, seed=27), 
                        param_grid = params2, 
                        scoring='accuracy',
                        n_jobs=4,
                        iid=False, 
                        cv=4)

In [ ]:
gsearch2.fit(X_train,y_train)

In [ ]:
gsearch2.best_params_, gsearch2.best_score_